We illustrate how we might write a class that indexes like numpy arrays and torch tensors.

In [1]:
from itertools import islice
from pathlib import Path
from urllib.request import urlretrieve
import gzip, pickle

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

if not path_gz.exists():
    urlretrieve(MNIST_URL, path_gz)

with gzip.open(path_gz, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

lst1 = list(x_train[0])
it = iter(lst1)
img = list(iter(lambda: list(islice(it, 28)), []))

Note especially the `__getitem__` method definition

In [2]:
class Matrix:
    def __init__(self, xs):
        self.xs = xs
    def __getitem__(self, idxs):
        return self.xs[idxs[0]][idxs[1]]


Observe that we have passed a tuple into `[...]`.

In [3]:
m = Matrix(img)
img[20][15], m[20,15]


(0.98828125, 0.98828125)